In [92]:
import pandas as pd 
from thefuzz import process

In [91]:
head_to_head_df = pd.read_csv('../data/raw/head_to_head_df.csv')

odds_df = pd.read_csv('../data/raw/historical_betting_odds_and_results.csv')
odds_df = odds_df[['HomeTeam','AwayTeam','FTHG','FTAG','B365CH','B365CD','B365CA','B365H','B365D','B365A']].copy()
odds_df['Match'] = odds_df['HomeTeam'] + ' ' + odds_df['FTHG'].astype(str) + '-' + odds_df['FTAG'].astype(str) + ' ' + odds_df['AwayTeam']

xg_df = pd.read_csv('../data/raw/xG.csv')
xg_df = xg_df[['Home','xG_Home','Score','xG_Away','Away']].copy()
xg_df['Match'] = xg_df['Home'] + ' ' + xg_df['Score'] + ' ' + xg_df['Away']
xg_df = xg_df.dropna()

### Liverpool FC

In [126]:
lfc_head_to_head_df = head_to_head_df[head_to_head_df.Match.str.contains('Liverpool')].drop_duplicates().copy()
lfc_head_to_head_df.reset_index(drop=True, inplace=True)

In [127]:
lfc_odds_df = odds_df[odds_df.Match.str.contains('Liverpool')].copy()
lfc_odds_df.reset_index(drop=True, inplace=True)

In [128]:
lfc_xg_df = xg_df[xg_df.Match.str.contains('Liverpool')].copy()

In [129]:
# Function to find the closest match
def find_best_match(row, choices):
    match, score, temp = process.extractOne(row, choices)
    return match if score >= 80 else None  # Use a threshold to avoid bad matches

In [130]:
# Apply the matching function
lfc_head_to_head_df['Best Match'] = lfc_head_to_head_df['Match'].apply(lambda x: find_best_match(x, lfc_odds_df['Match']))

In [131]:
# Merge the two DataFrames on the matched names
lfc_df = lfc_head_to_head_df.merge(lfc_odds_df, left_on='Best Match', right_on='Match', how='left', suffixes=('_a', '_b'))

# Drop the intermediate 'Best Match' column if not needed
lfc_df = lfc_df.drop(columns=['Best Match'])

In [132]:
lfc_df['Best Match'] = lfc_df['Match_a'].apply(lambda x: find_best_match(x, lfc_xg_df['Match']))

In [133]:
# Merge the two DataFrames on the matched names
lfc_df = lfc_df.merge(lfc_xg_df, left_on='Best Match', right_on='Match', how='left', suffixes=('_a', '_b'))

# Drop the intermediate 'Best Match' column if not needed
lfc_df = lfc_df.drop(columns=['Best Match'])

In [135]:
lfc_df.drop(columns=['Match_a','Match_b','Team1','Team2','HomeTeam','AwayTeam','FTHG','FTAG'],inplace=True)
lfc_df = lfc_df[['Match','Home','Score','Away','Wins1','Draws','Wins2','B365CH','B365CD','B365CA','B365H','B365D','B365A','xG_Home','xG_Away']]

In [136]:
lfc_df.to_csv('../data/processed/lfc_df.csv', index=False)

In [138]:
lfc_df.head()

,Match,Home,Score,Away,Wins1,Draws,Wins2,B365CH,B365CD,B365CA,B365H,B365D,B365A,xG_Home,xG_Away
0,Chelsea 1–1 Liverpool,Chelsea,1–1,Liverpool,10,15,11,3.1,3.60,2.15,2.90,3.40,2.38,1.4,1.3
1,Liverpool 3–1 Bournemouth,Liverpool,3–1,Bournemouth,15,1,2,1.2,7.50,12.00,1.20,7.00,13.00,3.0,1.3
2,Newcastle Utd 1–2 Liverpool,Newcastle Utd,1–2,Liverpool,4,6,14,2.2,3.75,3.10,2.20,3.75,3.00,2.0,0.9
3,Liverpool 3–0 Aston Villa,Liverpool,3–0,Aston Villa,14,3,6,1.6,4.75,4.75,1.65,4.50,4.50,2.5,0.7
4,Chelsea 1–1 Liverpool,Chelsea,1–1,Liverpool,4,1,13,3.1,3.60,2.15,2.90,3.40,2.38,1.4,1.3
